# Introduction
Hi I tried to find a solution using the ant method I saw in a Sebastian Lague video (https://www.youtube.com/watch?v=X-iSQQgOd1A&t=207s).

It consists in taking a pack of ants, making them start in a random city (or here permutation), each of which looks at the distance that separates it from the next cities. They then randomly choose a city with greater luck if the city is closer. They continue like this until they get a path. The path then obtains a score which depends on the length of the path, the shorter the path the better, which allows to increase or reduce the power of the pheromones. Each possible movement between two cities has a pheromone level. The higher the level, the more likely the ants have to take this route.

This way the ants remember the most efficient paths and try to optimize them. But it is, of course, a local minimum.

There are two class in my code, ant which represents an ants and map which represents the land on which the ants are.

It's more or less my first data project so I hope it's okay. It is far from being optimized. I had ideas to use numba cuda, it can be done quite easily with an incredible performance improvement I think. Since all the ants would work in parallel. But I'll leave it there for now. I also had ideas to optimize the stars and many other things but I have to focus on my google data analytics certification. To give an idea this gives me a score of about 4000 for 30s of computation time.

# Functions

In [ ]:
import random
import pandas as pd
import itertools
import numpy as np
import torch
import datetime
import torch.nn.functional as F
from tqdm.notebook import tqdm

In [ ]:
# Return three list containing all the combinations that each map must use
def clean(permus, fraq_1=1/3, fraq_2=1/2):
    start = permus[permus.Permutation.str[:2] =='🎅🤶']
    other = permus[permus.Permutation.str[:2] !='🎅🤶'].reset_index(drop=True)

    # Fais 3 liste qui contient les differentes permutations
    permus4str_1 = start.copy()
    permus4str_2 = start.copy()
    permus4str_3 = start.copy()

    add_to_1 = other.sample(frac=fraq_1)
    other = other.drop(add_to_1['Permutation'].index.tolist())
    add_to_2 = other.sample(frac=fraq_2)
    other = other.drop(add_to_2['Permutation'].index.tolist())
    add_to_3 = other.copy()

    permus4str_1 = pd.concat([permus4str_1, add_to_1])
    permus4str_1 = permus4str_1.sort_index()
    permus4str_2 = pd.concat([permus4str_2, add_to_2])
    permus4str_2 = permus4str_2.sort_index()
    permus4str_3 = pd.concat([permus4str_3, add_to_3])

    city_1, city_2, city_3 = [], [], []

    for string in permus4str_1['Permutation'].tolist():
        city_1.append(change(string))

    for string in permus4str_2['Permutation'].tolist():
        city_2.append(change(string))

    for string in permus4str_3['Permutation'].tolist():
        city_3.append(change(string))

    return city_1, city_2, city_3

# Return the distance matrix with only the distances of the cities in the list city
def get_dist_mat(dist_mat, city):
    to_export = pd.DataFrame(columns=city, index=city)
    for c in city:
        to_export.loc[c] = dist_mat.loc[c]

    to_export = to_export.to_numpy()
    to_export[to_export==0] = 99999

    return to_export

# Check if the submission the good
def check_if_good(a):
    is_false = False
    error_count_start = 0
    error_count_other = 0
    # Check if the submission is valid
    wildcard = pd.read_csv('../input/santa-2021/wildcards.csv')
    permus = pd.read_csv('../input/santa-2021/permutations.csv')
    start = permus[permus.Permutation.str[:2] =='🎅🤶']
    other = permus[permus.Permutation.str[:2] !='🎅🤶'].reset_index(drop=True)

    # If all the combinations beginning with 🎅🤶 are in the three submissions
    for permu in start['Permutation'].tolist():
        if permu in a[0]:
            if permu in a[1]:
                if permu in a[2]:
                    continue
        # For wildcards
        permus_for_wild = wildcard[wildcard['Permutation']==permu].index.values
        in_string = False
        for p in permus_for_wild:
            if wildcard.at[p, 'Factor'] in a[0]: in_string = True
            if wildcard.at[p, 'Factor'] in a[1]: in_string = True
            if wildcard.at[p, 'Factor'] in a[2]: in_string = True
        if in_string: continue
        is_false = True
        #print("Not all 🎅🤶, missing:", permu)
        error_count_start += 1

    # If all the combinations are in the submissions
    for permu in other['Permutation'].tolist():
        if permu in a[0]: continue
        if permu in a[1]: continue
        if permu in a[2]: continue

        # For wildcards
        permus_for_wild = wildcard[wildcard['Permutation']==permu].index.values
        in_string = False
        for p in permus_for_wild:
            if wildcard.at[p, 'Factor'] in a[0]: in_string = True
            if wildcard.at[p, 'Factor'] in a[1]: in_string = True
            if wildcard.at[p, 'Factor'] in a[2]: in_string = True
        if not in_string:
            is_false = True
            #print("Not all the combinaison, missing:", permu)
            error_count_other += 1

    # If there are 2 stars or less
    if a[0].count('🌟') > 2:
        print("Too many stars in string: 0")
        return False
    if a[1].count('🌟') > 2:
        print("Too many stars in string: 1")
        return False
    if a[2].count('🌟') > 2:
        print("Too many stars in string: 2")
        return False

    if is_false:
        print("Nombre d'erreur start:", error_count_start, ', other', error_count_other)
        return False
    return True

# Turn icons into numbers
def change(a):
    a = a.replace("🎅", "1")
    a = a.replace('🤶', "2")
    a = a.replace('🦌', "3")
    a = a.replace('🧝', "4")
    a = a.replace('🎄', "5")
    a = a.replace('🎁', "6")
    a = a.replace('🎀', "7")
    return a


def rechange(a):
    a = a.replace("1", "🎅")
    a = a.replace("2", '🤶')
    a = a.replace("3", '🦌')
    a = a.replace("4", '🧝')
    a = a.replace("5", '🎄')
    a = a.replace("6", '🎁')
    a = a.replace("7", '🎀')
    return a

# Save the solution
def save_schedule(sub, permus):
    if check_if_good(sub):
        print("Submission valid!")
        print('Final score:', max([len(sub[0]), len(sub[1]), len(sub[2])]))
        sub_df = pd.DataFrame()
        sub_df['schedule'] = sub
        sub_df.to_csv('submission.csv', index=False)
        print(sub_df.head())

# Removes cities already used by chance by a previous map
def clear_already_in(a, b, permus):
    in_b = []
    for p in permus['Permutation'].tolist():
        p = change(p)
        if p in b:
            in_b.append(p)

    for p in in_b:
        if p in a:
            a.remove(p)
    return a

In [ ]:
# Optimisation stuff, thx YOSSHI999
def find_strings_perms(strings, verbose=False):
    found_perms = []
    perms = list(map(lambda p: "".join(p), itertools.permutations("1234567")))
    for s in strings:
        found_perms.append([])
        for i in range(len(s)-6):
            p = s[i:i+7]
            if p in perms:
                found_perms[-1].append(p)
    if verbose:
        lens = [len(_) for _ in  found_perms]
        print(f'There are {lens} permutations in strings, {sum(lens)} in total.')
        lens = [len(set(_)) for _ in  found_perms]
        print(f'There are {lens} unique permutations in strings, {sum(lens)} in total.')
    return found_perms


def rebalance_perms(strings_perms, verbose=False):
    # convert to dicts for fast lookup and to keep permutations order
    strings_perms = [dict.fromkeys(_) for _ in strings_perms]
    for p in strings_perms[0].copy():  # iterate over the copy to allow modification during iteration
        if p[:2] != "12" and (p in strings_perms[1] or p in strings_perms[2]):
            strings_perms[0].pop(p)
    for p in strings_perms[1].copy():
        if p[:2] != "12" and p in strings_perms[2]:
            strings_perms[1].pop(p)
    if verbose:
        lens = [len(_) for _ in  strings_perms]
        print(f'There are {lens} permutations left in strings after rebalancing, {sum(lens)} in total.')
    return [list(_) for _ in strings_perms]


def star_optimization(a):
    symbols = "🎅🤶🦌🧝🎄🎁🎀"
    words = a

    schedules = []
    schedules.append(change(a[0]))
    schedules.append(change(a[1]))
    schedules.append(change(a[2]))

    perms = list(map(lambda p: "".join(p), itertools.permutations("1234567")))
    found_perms = find_strings_perms(schedules)
    balanced_perms = rebalance_perms(found_perms)
    perm2id = {p: i for i, p in enumerate(perms)}
    perms_arr = np.array([list(map(int, p)) for p in perms])

    perms_onehot = np.eye(7)[perms_arr-1, :].transpose(0, 2, 1)
    assert np.allclose(perms_onehot[:,0,:].astype(np.int64), (perms_arr == 1).astype(np.int64))

    left = perms_onehot[perm2id["1234567"]]
    right = perms_onehot[perm2id["5671234"]]
    matches = F.conv2d(
        F.pad(torch.Tensor(left[None, None, :, :]), (7, 7)),
        torch.Tensor(right[None, None, :, :]),
        padding="valid"
    ).numpy().reshape(-1)
    must_match_left2right = np.array([-1, -1, -1, -1, -1, -1, -1, 7, 6, 5, 4, 3, 2, 1, 0])
    must_match_right2left = np.array([0, 1, 2, 3, 4, 5, 6, 7, -1, -1, -1, -1, -1, -1, -1])
    cost_ifmatch = np.array([7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7])

    M = F.conv2d(
        F.pad(torch.Tensor(perms_onehot[:, None, :, :]), (7, 7)),
        torch.Tensor(perms_onehot[:, None, :, :]),
        padding="valid"
    ).squeeze().numpy()

    must_match_left2right = np.array([-1, -1, -1, -1, -1, -1, -1, 7, 6, 5, 4, 3, 2, 1, 0])
    must_match_left2right_wild = np.array([-1, -1, -1, -1, -1, -1, -1, 6, 5, 4, 3, 2, 1, 0, 0])

    cost_ifmatch = np.array([7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7])

    costMat = np.where(M == must_match_left2right, cost_ifmatch, np.inf).min(axis=-1).astype(np.int8)
    costMatWild = np.minimum(costMat, np.where(M == must_match_left2right_wild, cost_ifmatch, np.inf).min(axis=-1)).astype(np.int8)

    nodes_list = []
    table_list = []

    for i in range(3):
        word = words[i]
        nodes = [perm2id[p] for p in balanced_perms[i]]

        table = np.zeros((len(nodes), 10), np.int64)
        table[0, :] = 7
        for i in range(1, len(nodes)):
            e = costMat[nodes[i-1], nodes[i]]
            ew = costMatWild[nodes[i-1], nodes[i]]
            table[i,0] = table[i-1,0] + e
            table[i,1] = min(table[i-1,1] + e, table[i-1,0] + ew)
            table[i,2] = min(table[i-1,2], table[i-1,1]) + e
            table[i,3] = min(table[i-1,3], table[i-1,2]) + e
            table[i,4] = min(table[i-1,4], table[i-1,3]) + e
            table[i,5] = min(table[i-1,5], table[i-1,4]) + e
            table[i,6] = min(table[i-1,6], table[i-1,5]) + e
            table[i,7] = min(table[i-1,7], table[i-1,6]) + e
            table[i,8] = min(table[i-1,8], table[i-1,7]) + e
            table[i,9] = min(table[i-1,9] + e, table[i-1,8] + ew)
        nodes_list.append(nodes)
        table_list.append(table)

    # backtrack
    new_words = []
    wilds = []
    for nodes, table in zip(nodes_list, table_list):
        ns = [perms[nodes[-1]]]
        track = np.argmin(table[-1])
        wild = []
        for i in range(len(nodes)-2, -1, -1):
            e = costMat[nodes[i], nodes[i+1]]
            ew = costMatWild[nodes[i], nodes[i+1]]
            if track == 0:
                ns.append(perms[nodes[i]][:e])
            elif track == 1:
                if table[i, 1] + e < table[i, 0] + ew:
                    ns.append(perms[nodes[i]][:e])
                else:
                    left = np.array(list(map(int, perms[nodes[i]][ew:])))
                    right = np.array(list(map(int, perms[nodes[i+1]][:-ew])))
                    mis = np.where(left != right)[0][0]
                    wild.append(table[i, track-1]-7+ew+mis)
                    ns.append(perms[nodes[i]][:ew])
                    track = track - 1
            elif 2 <= track <= 8:
                if table[i, track] >= table[i, track-1]:
                    track = track - 1
                ns.append(perms[nodes[i]][:e])
            elif track == 9:
                if table[i, 9] + e < table[i, 8] + ew:
                    ns.append(perms[nodes[i]][:e])
                else:
                    ns.append(perms[nodes[i]][:ew])
                    left = np.array(list(map(int, perms[nodes[i]][ew:])))
                    right = np.array(list(map(int, perms[nodes[i+1]][:-ew])))
                    mis = np.where(left != right)[0][0]
                    wild.append(table[i, track-1]-7+ew+mis)
                    track = track - 1
            else:
                assert False
        assert track == 0
        wilds.append(wild)
        nsw = list("".join(ns[::-1]))
        for w in wild:
            nsw[w] = "*"
        new_words.append("".join(nsw))

    print("Optimisation done")
    print("score: ", max(map(len, words)), "->", max(map(len, new_words)))
    sub = [a.translate(str.maketrans("1234567*", symbols+"🌟")) for a in new_words]
    if check_if_good(sub): print("Optimization validated")

    return sub

# Class

In [ ]:
# The ant class
class Ant:
    def __init__(self, map):
        self.map = map
        self.visited = np.ones((self.map.nb_city,), dtype=bool)  # Boolean of if the city has been visited
        self.position_id = self.map.city.index(random.choice(self.map.city))  # Id of current position
        self.visited[self.position_id] = 0
        self.dist_trav = 7  # Distance traveled
        self.path = self.map.city[self.position_id] # What we are looking for
        self.path_id = [0] * self.map.nb_city # History of the path for pheromones
        self.path_id[0] = self.position_id
        self.path_act_id = 1

    def update(self):
        dis, phero = self.map.dist_mat[self.position_id, :], self.map.pheros[self.position_id, :]
        dis[dis < 0] = self.map.inf_dist

        weight = (self.visited * phero) / ((dis * 2) ** self.map.coef_dist_puiss)

        next_city_id = random.choices(range(self.map.nb_city), weights=weight, k=1)[0]

        self.path += self.map.city[next_city_id][(7-int(dis[next_city_id])):]
        self.path_id[self.path_act_id] = next_city_id
        self.path_act_id += 1

        self.dist_trav += self.map.dist_mat[self.position_id, next_city_id]
        self.position_id = next_city_id
        self.visited[self.position_id] = 0

    def find_path(self):
        for i in range(self.map.nb_city-1):
            self.update()

    def update_pheros(self):
        x = 100
        coef = -50
        for i in range(x):
            if self.dist_trav <= (self.map.dist_min + self.map.dist_delta * (i / x)):
                coef += 1
        for i in range(len(self.path_id)-1):
            self.map.pheros[self.path_id[i], self.path_id[i+1]] += coef

In [ ]:
# The map class
class Map:
    # Settings
    phero_start_fact = 1000
    inf_dist = 9999

    def __init__(self, city, dist_mat, name, coef_dist_puiss=20,
                 nb_ant=20, nb_turn=100, do_print=True):
        self.city = city # List of city to travel
        self.name = name # Name of the map
        self.nb_ant = nb_ant # Number of ants
        self.nb_turn = nb_turn # Number of turn to do
        self.dist_mat = dist_mat # Distance matrix

        self.pheros = np.ones(self.dist_mat.shape) * self.phero_start_fact # Pheromones matrix

        self.shortest_path = ""
        self.shortest_dist = 9999999999999
        self.dist_min = 0
        self.dist_max = 0
        self.dist_delta = 0
        self.nb_city = len(city)

        self.coef_dist_puiss = coef_dist_puiss # Coef to use in ant.update, 5-10 is good
        self.ants = [] # List of ant
        for i in range(self.nb_ant):
            self.ants.append(Ant(self))

        self.pbar = None # The progress bar

    def update(self):
        ants_dist = [0] * len(self.ants)
        i = 0
        for ant in self.ants:
            ant.find_path()
            dis = ant.dist_trav
            ants_dist[i] = dis
            if dis < self.shortest_dist:
                self.shortest_dist = dis
                self.shortest_path = ant.path
            i+=1
            self.pbar.update(1)
        self.dist_min = min(ants_dist)
        self.dist_max = max(ants_dist)
        self.dist_delta = self.dist_max - self.dist_min
        for ant in self.ants:
            ant.update_pheros()
            ant.__init__(self)

    def start(self):
        # Start the simulation
        with tqdm(total=self.nb_turn*self.nb_ant, desc=self.name,unit="ant") as pbar:
            self.pbar = pbar
            for i in range(self.nb_turn):
                self.update()


# Main

In [ ]:
#-------------------------------------------------------------------------------------------------
print("Init...")
np.seterr(divide='ignore', invalid='ignore')

#-------------------------------------------------------------------------------------------------
print("Import...")

permus = pd.read_csv("../input/santa-2021/permutations.csv")
dist_mat = pd.read_csv("../input/santa-2021/distance_matrix.csv")
dist_mat.set_index('Permutation', inplace=True)
dic = {}
for c in dist_mat.columns:
    dic[c] = change(c)
dist_mat.rename(dic, axis=1, inplace=True)
dist_mat.rename(dic, axis=0, inplace=True)
other = permus[permus.Permutation.str[:2] !='🎅🤶'].reset_index(drop=True)

#-------------------------------------------------------------------------------------------------
print("Clean...")

city_1, city_2, city_3 = clean(permus, fraq_1=0.28, fraq_2=0.42)

#-------------------------------------------------------------------------------------------------
print("Calcul...")
nb_ant = 2
nb_turn = 5
coef_dist_puiss = 10

print("The calculation will take approximately " + str(datetime.timedelta(seconds=(1.2 * 3 * nb_ant * nb_turn))))
print('Start for nb_ant:', nb_ant, ', nb_turn', nb_turn, )

dist_mat_map_1 = get_dist_mat(dist_mat, city_1)
map_1 = Map(city_1, dist_mat_map_1, "map_1", nb_ant=nb_ant, nb_turn=nb_turn,
            coef_dist_puiss=coef_dist_puiss, do_print=False)
map_1.start()
print("map_1 Done, score:", map_1.shortest_dist, 'Nb_city:', len(city_1))

city_2 = clear_already_in(city_2, map_1.shortest_path, other)
city_3 = clear_already_in(city_3, map_1.shortest_path, other)

dist_mat_map_2 = get_dist_mat(dist_mat, city_2)
map_2 = Map(city_2, dist_mat_map_2, "map_2", nb_ant=nb_ant, nb_turn=nb_turn,
            coef_dist_puiss=coef_dist_puiss, do_print=False)
map_2.start()
print("map_2 Done, score:", map_2.shortest_dist, 'Nb_city:', len(city_2))

city_3 = clear_already_in(city_3, map_2.shortest_path, other)

dist_mat_map_3 = get_dist_mat(dist_mat, city_3)
map_3 = Map(city_3, dist_mat_map_3, "map_3", nb_ant=nb_ant, nb_turn=nb_turn,
            coef_dist_puiss=coef_dist_puiss, do_print=False)
map_3.start()
print("map_3 Done, score:", map_3.shortest_dist, 'Nb_city:', len(city_3))

#-------------------------------------------------------------------------------------------------
print("Optimise...")

paths = [rechange(map_1.shortest_path), rechange(map_2.shortest_path), rechange(map_3.shortest_path)]
paths = star_optimization(paths)

#-------------------------------------------------------------------------------------------------
print("Check...")

save_schedule(paths, permus)

#-------------------------------------------------------------------------------------------------
print('Done!')